In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca
from robot import *
from helper_fns import *

In [2]:
par = {'pos': [0, 0, 0.0],
       'stiff': [0, 0, 3e4],
        'rest':[0, 0, 0.7]}
attrs = yaml_load('config/attrs.yaml')
c = Contact('c/', par, ['rest'], attrs=attrs)
rob = LinearizedRobot('config/franka.urdf', subsys = [c], attrs = attrs, visc_fric = 100)

Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['c/']


In [4]:
from observer import EKF

rob.build_step(0.002)
rob.build_linearized()
A, C, n, y = rob.linearized(np.zeros(rob.nx), np.zeros(7), rob.inv_mass_fn(np.zeros(7)))
ekf = EKF(rob)
#%timeit ekf.step(0.01*np.ones(7), np.zeros(7), np.eye(7))
# 3x speedup from JITting the EKF step
for _ in range(100):
    ekf.step(0.01*np.ones(7), np.zeros(7))
print(ekf.get_ext_state())

{'q': DM([0.01, 0.00999999, 0.01, 0.01, 0.01, 0.01, 0.01]), 'dq': DM([1.10829e-07, -4.44406e-06, 1.12288e-07, 1.93953e-06, 9.27662e-08, -5.86775e-08, -5.6605e-09]), 'c/rest': DM([0, 0, 0.926045]), 'p': DM([0.0921816, 0.00267534, 0.926045]), 'R': DM(
[[0.99975, 0.0199987, 0.00999385], 
 [0.0199957, -0.9998, 0.000399927], 
 [0.00999985, -0.000199993, -0.99995]]), 'dx': DM([-3.18132e-06, -9.16126e-09, 4.17261e-07]), 'c/F': DM([0, 0, -2.58093e-05]), 'c/disp': DM([0.0921816, 0.00267534, 8.6031e-10]), 'c/n': DM([0, 0, 1]), 'c/x': DM([0.0921816, 0.00267534, 0.926045])}


In [14]:
from impedance_controller import ImpedanceController
from robot import Robot
imp = ImpedanceController(input_vars = [])
rob = Robot('config/franka.urdf', ctrl = imp)
rob.build_step(0.05)
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
state = {'q':q, 'dq':dq, 'M_inv':rob.inv_mass_fn(q), 
         'imp_stiff':[200, 200, 200], 'imp_rest':[0.0,0,1.1]}
#%timeit step(**state)

print(rob.step)
for _ in range(200):
    res = rob.step.call(state)
    state['q'] = res['q']
    state['dq'] = res['dq']
    #state['M_inv'] = rob.inv_mass_fn(state['q'])
    res_dict = rob.get_ext_state(state)
    print(res_dict['p'])

#print(rob.inv_mass_fn(np.zeros(7)))


Building robot model from config/franka.urdf with TCP fr3_link8
  with control imp_ctrl
step:(q[7],dq[7],M_inv[7x7],imp_stiff[3],imp_rest[3])->(q[7],dq[7],cost) SXFunction
[0.0847121, -6.98315e-05, 0.929109]
[0.0774152, -0.000146098, 0.933512]
[0.0666194, -0.000208012, 0.938557]
[0.0532923, -0.000248113, 0.943906]
[0.0385701, -0.000265222, 0.949357]
[0.023611, -0.000261803, 0.954818]
[0.00949506, -0.000242167, 0.960297]
[-0.00284944, -0.000211006, 0.965875]
[-0.0126983, -0.000172398, 0.971666]
[-0.0195636, -0.000129386, 0.977775]
[-0.0232085, -8.40769e-05, 0.984257]
[-0.0236492, -3.80888e-05, 0.991108]
[-0.0211446, 6.88827e-06, 0.998261]
[-0.016174, 4.86567e-05, 1.00561]
[-0.00940166, 8.4373e-05, 1.01303]
[-0.00162774, 0.000110792, 1.02042]
[0.00627467, 0.000124853, 1.02771]
[0.0134309, 0.000124453, 1.0349]
[0.0190402, 0.000109182, 1.042]
[0.0224477, 8.07725e-05, 1.04904]
[0.0232077, 4.3091e-05, 1.05605]
[0.0211328, 1.60393e-06, 1.06302]
[0.0163237, -3.75806e-05, 1.06988]
[0.00917639, 